In [ ]:
class Carta :
    """
    Clase Carta; representa un carta del mazo. Esta clase está pensada para ser usada dentro de la clase Mazo; sin embargo, se pueden generar cartas individuales.

    Args:
        palo (str): 'Corazones', 'Diamantes', 'Tréboles', 'Picas'
        valor (str): Valor de la carta ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')
    """

    def __init__(self, palo, valor):    # Método;: Inicio de objeto.
        self.palo = palo    # Atributo (str): Palo al que pertenece la carta.
        self.valor = valor  # Atributo (str): Valor nominal de la carta.

class Mazo :
    """
    Clase Mazo, representa un conjunto de cartas.
    """
    def __init__(self): # Método: Inicio del objeto.
        self.mazo = self.generar_cartas()   # Atributo (list): Lista de cartas; se genera automáticamente al iniciar el objeto llamando el método generar_cartas()
        self.tamaño = len(self.mazo)    # Atributo (int): Tamaño del mazo.

    def generar_cartas(self) :
        """
        Método para generar las 52 cartas del mazo. Esta función se auto-llama al crear el objeto mazo.

        Returns:
            mazo (object): Conjunto de 52 cartas.
        """
        palos = ('Corazones', 'Diamantes', 'Tréboles', 'Picas') # Tupla de palos
        valores = ('2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K', 'A')    # Tupla de valores

        mazo = []   # Lista que almacena las cartas generadas.
        for palo in palos :
            for valor in valores :
                mazo.append(Carta(palo,valor))
    
        return mazo

    def revolver(self) :
        """
        Método para revolver las 52 cartas del mazo.
        """
        import random   # Importa el módulo random usado para revolver las cartas.
        random.shuffle(self.mazo)

    def repartir(self, jugadores: list, cartas_por_mano: int) :
        """
        Método para repartir las cartas entre los jugadores seleccionados. El método interactúa directamente con la clase Jugador.

        Args:
            jugadores (list): Lista de jugadores.
            cartas_por_mano (int): Cantidad de cartas a repartir a cada jugador.
        """
        num_jugadores = len(jugadores)

        manos = []
        for i in range(0,num_jugadores) :
            mano = []
            for j in range(0,cartas_por_mano) :
                mano.append(self.mazo[i*cartas_por_mano+j])
            manos.append(mano)    

        for i in range(0,num_jugadores) :
            jugadores[i].número = i+1
            jugadores[i].mano = manos[i]

class Jugador :
    """
    Clase Jugador; representa un jugador que puede participar en el juego.

    Args:
        nombre (str): Nombre del jugador.
    """
    def __init__(self, nombre):
        self.nombre = nombre
        self.número = 0
        self.mano = []

    def mostrar_mano(self) :
        """
        Método que muestra los atributos "palo" y "valor" de cada objeto carta en la mano del jugador.

        Returns:
            mano (list) : Lista de cartas expresada como f"{valor} de {palo}".
        """
        mano = []
        for carta in self.mano :
            mano.append(f"{carta.valor} de {carta.palo}")
        return mano

In [ ]:
def simular_póquer(mazo: object, jugadores: list, iteraciones: int, mano_a_buscar: str, regenerar_mazo=False) -> dict :
    """
    Función que permite repetir n veces la repartición y búsqueda de un tipo de mano específica de póquer.

    Args:
        mazo (object): Mazo.
        jugadores (list): Lista de jugadores a participar en el juego.
        iteraciones (int): Cantidad de veces a repetir la simulación.
        mano_a_buscar (str): Opciones "Flor Imperial" y "Mano de muerto"

    Returns:
        Dictionary: Diccionario que contiene la cantidad total de juegos simulados ('Total jugado'), el total de partidas ('Jugadas encontradas') que cumplen con el criterio de búsqueda (flores imperiales o manos de muerto) y la probabilidad de que ocurra el tipo de partida buscada ('Probabilidad').
    """
    
    def buscar_flores_imperiales(jugador: object) -> bool :
        """
        Función que permite buscar flores imperiales dentro de la mano del jugador.

        Args:
            jugador (object): Jugador.
        """
        valores_imperiales = ('10', 'J', 'Q', 'K', 'A')
    
        palos = [carta.palo for carta in jugador.mano]
        valores = [carta.valor for carta in jugador.mano]

        if len(set(palos)) == 1 :
            indicador = 0
            for valor in valores :
                if valor in valores_imperiales :
                    indicador += 1
            return indicador == 5
        else :
            return False

    def buscar_mano_muerto(jugador: object) -> bool:
        """
        Función que permite buscar la mano del muerto dentro de la mano de un jugador.

        Args:
            jugador (object): Jugador.
        """
        valor_muerto = ('8','A')
        palo_muerto = ('Tréboles', 'Picas')

        contador = 0
        for carta in jugador.mano :
            if carta.valor in valor_muerto and carta.palo in palo_muerto :
                contador += 1
        
        return contador == 4
        
    if len(jugadores) != len(set(jugadores)) :
        raise ValueError("Hay jugadores repetidos en la lista.")
    if len(jugadores) > 52//5 :
        raise ValueError(f"La cantidad máxima de jugadores es {52//5}.")

    match mano_a_buscar.lower() :
        case "flor imperial" :
            juego = buscar_flores_imperiales
        case "mano de muerto" :
            juego = buscar_mano_muerto

    jugadas_totales = 0
    jugadas_clave = 0

    for i in range(0,iteraciones) :
        mazo.revolver()
        mazo.repartir(jugadores, 5)

        for jugador in set(jugadores) :
            if juego(jugador) :
                jugadas_clave += 1
                print(f"(Juego# {jugadas_totales+1}) {jugador.nombre}: {jugador.mostrar_mano()}")
    
        jugadas_totales += 1

        if regenerar_mazo :
            mazo.generar_cartas()

    return {
        "Total jugado" :  jugadas_totales,
        "Jugadas encontradas" : jugadas_clave,
        "Probabilidad": f"{jugadas_clave / jugadas_totales * 100} %"
    }

In [ ]:
Juan = Jugador("Juan")
Pedro = Jugador("Pedro")
Pablo = Jugador("Pablo")
Paco = Jugador("Paco")
Jesús = Jugador("Jesús")
Ignacio = Jugador("Ignacio")
Fulanito = Jugador("Fulanito")
Sultanito = Jugador("Sultanito")
Menganito = Jugador("Menganito")
Cosme = Jugador("Cosme Fulanito")

mazo = Mazo()

In [ ]:
simular_póquer(mazo, [Juan, Pedro, Pablo, Paco], 1000000, "Flor imperial")

In [ ]:
def buscar_flores_imperiales_2_0(jugador: object) :
    """
    Función que permite buscar flores imperiales dentro de la mano del jugador.

    Args:
        jugador (object): Jugador.
    """
    palos = ('Corazones', 'Diamantes', 'Tréboles', 'Picas')
    valores_imperiales = ('10', 'J', 'Q', 'K', 'A')
    cartas_jugador = {}

    for palo in palos :
        cartas_jugador.update({palo:[]})

    for carta in jugador.mano :
        cartas_jugador[carta.palo].append(carta)

    indicador_imperial = 0
    for key in cartas_jugador :
        if len(cartas_jugador[key]) >= 5 :
            for card in cartas_jugador[key] :
                if card.valor in valores_imperiales :
                    indicador_imperial += 1

    if indicador_imperial >= 5 :
        return True
    else :
        return False